In [1]:
!pip install timm==0.3.2
!pip install torch==1.8.1 torchvision==0.9.1 torchaudio===0.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 244 kB 4.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 2.3 kB/s 
     |████████████████████████████████| 17.4 MB 24.1 MB/s 
     |████████████████████████████████| 1.9 MB 40.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1+cu113
    Uninstalling torchaudio-0.12.1+cu113:
      Successfully uninstalled torchaudio-0.12.1+cu113
ERROR: pip's depen

In [ ]:
# For Paperspace to run PyTorch
!pip install ipywidgets

In [ ]:
# NOTE: Mounted coffee-beans dataset (in Paperspace)
!ls /datasets/coffee-beans

# Unzip dataset
!unzip "/datasets/coffee-beans/beans-data-20220903T002815Z-001.zip"

!ls

# Absolute data directory path
data_dir = "/notebooks/convit/beans-data/"

beans-data-20220903T002815Z-001.zip
Archive:  /datasets/coffee-beans/beans-data-20220903T002815Z-001.zip
replace beans-data/val/Dark/dark (76).png? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
LICENSE      beans-data   engine.py  requirements.txt	   utils.py
README.md    convit.py	  main.py    run_with_submitit.py
__pycache__  datasets.py  models.py  samplers.py


In [3]:
!rm -rf /content/convit/
!git clone https://github.com/GoelBiju/convit.git

Cloning into 'convit'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 64 (delta 30), reused 48 (delta 17), pack-reused 0
Unpacking objects: 100% (64/64), done.


In [4]:
%cd convit
!pwd

/content/convit
/content/convit


In [5]:
# NOTE: For Google Colaboratory
# Data source
from google.colab import drive

# Importing the dataset (stored in Google Drive)
drive.mount("/content/drive", force_remount=True)

!ls "/content/drive/MyDrive/PROJ518/Cotton Dataset/data"
data_dir = "/content/drive/MyDrive/PROJ518/Cotton Dataset/data"

Mounted at /content/drive
test  train  val


In [6]:
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
from torch.autograd import Variable

import os

# "models.py" in convit repository
from models import convit_tiny, convit_base
from timm.models import create_model
# from mydatasets import SeedlingData

from PIL import Image
from torch.utils import data
from torchvision import transforms as T
from sklearn.model_selection import train_test_split

In [7]:
os.environ['CUDA_VISIBLE_DEVICE'] = '0'

torch.cuda.set_device(0)

# Get cpu or gpu device for training
# device = "cuda" if torch.cuda.is_available() else "cpu"
# print("Using {} device".format(device))

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Device: ", DEVICE)


# Set global parameters
modellr = 1e-4
BATCH_SIZE = 32 # 32
EPOCHS = 25 # 50

Device:  cuda:0


In [8]:
# Data preprocessing

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.4843, 0.5642, 0.3729], [0.1966, 0.1852, 0.1913])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.4843, 0.5642, 0.3729], [0.1966, 0.1852, 0.1913])
])

In [9]:
# Labels = {'Black-grass': 0, 'Charlock': 1, 'Cleavers': 2, 'Common Chickweed': 3,
#           'Common wheat': 4, 'Fat Hen': 5, 'Loose Silky-bent': 6, 'Maize': 7, 'Scentless Mayweed': 8,
#           'Shepherds Purse': 9, 'Small-flowered Cranesbill': 10, 'Sugar beet': 11}

# Labels = {
#     'Dark': 0,
#     'Green': 1,
#     'Light': 2,
#     'Medium': 3
# }

Labels = {
    'healthy': 0,
    'leaf_curl': 1,
}
 

class SeedlingData(data.Dataset):
    def __init__(self, root, transforms=None, train=True, test=False):
        """
        Main objective: to obtain the addresses of all pictures and divide the data according to training, verification and test
        """
        self.test = test
        self.transforms = transforms

        if self.test:
            # imgs = [os.path.join(root, img) for img in os.listdir(root)]
            # print(imgs)
            # self.imgs = imgs

            imgs_labels = [os.path.join(root, img) for img in os.listdir(root)]
            imgs = []
            for img_label in imgs_labels:
                for imgname in os.listdir(img_label):
                    imgpath = os.path.join(img_label, imgname)
                    imgs.append(imgpath)
            self.imgs = imgs
        else:
            imgs_labels = [os.path.join(root, img) for img in os.listdir(root)]
            imgs = []
            for img_label in imgs_labels:
                for imgname in os.listdir(img_label):
                    imgpath = os.path.join(img_label, imgname)
                    imgs.append(imgpath)
            trainval_files, val_files = train_test_split(imgs, test_size=0.3, random_state=42)
            if train:
                self.imgs = trainval_files
            else:
                self.imgs = val_files

    def __getitem__(self, index):
        """
        Returns the data of one picture at a time
        """
        img_path = self.imgs[index]
        img_path=img_path.replace("\\",'/')
        if self.test:
          labelname = img_path.split('/')[-2]
          label = Labels[labelname]
          # label = -1
        else:
            labelname = img_path.split('/')[-2]
            label = Labels[labelname]
        data = Image.open(img_path).convert('RGB')
        data = self.transforms(data)
        return data, label

    def __len__(self):
        return len(self.imgs)


In [10]:
# Create train and test datasets
print("Training source: ", os.path.join(data_dir, "train"))
dataset_train = SeedlingData(os.path.join(data_dir, "train"), transforms=transform, train=True)
dataset_test = SeedlingData(os.path.join(data_dir, "val"), transforms=transform_test, train=False)

# Read data
print(dataset_train.imgs)

# Import data
# TODO: sampler for train? RASampler, DistributedSampler/Random Sampler
train_loader = torch.utils.data.DataLoader(dataset_train, 
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset_test, 
                                          batch_size=BATCH_SIZE, # 1.5 * BATCH_SIZE? 
                                          shuffle=False)


# TODO: Implement MixUp/CutMix...
# Mixup is a generic and straightforward data augmentation principle. 
# In essence, mixup trains a neural network on convex combinations of pairs of 
# examples and their labels. By doing so, mixup regularizes the neural network to 
# favor simple linear behavior in-between training examples.



Training source:  /content/drive/MyDrive/PROJ518/Cotton Dataset/data/train
['/content/drive/MyDrive/PROJ518/Cotton Dataset/data/train/leaf_curl/aug_lc_48_9888.png', '/content/drive/MyDrive/PROJ518/Cotton Dataset/data/train/leaf_curl/aug_lc_82_2507.png', '/content/drive/MyDrive/PROJ518/Cotton Dataset/data/train/healthy/healthy_421.jpg', '/content/drive/MyDrive/PROJ518/Cotton Dataset/data/train/healthy/healthy_907.jpg', '/content/drive/MyDrive/PROJ518/Cotton Dataset/data/train/healthy/healthy_498.jpg', '/content/drive/MyDrive/PROJ518/Cotton Dataset/data/train/healthy/healthy_404.jpg', '/content/drive/MyDrive/PROJ518/Cotton Dataset/data/train/healthy/healthy_622.jpg', '/content/drive/MyDrive/PROJ518/Cotton Dataset/data/train/healthy/healthy_632.jpg', '/content/drive/MyDrive/PROJ518/Cotton Dataset/data/train/leaf_curl/aug_lc_99_610.png', '/content/drive/MyDrive/PROJ518/Cotton Dataset/data/train/healthy/healthy_698.jpg', '/content/drive/MyDrive/PROJ518/Cotton Dataset/data/train/leaf_curl/au

In [11]:
# Set the model

# Instantiate the model and move to the GPU
# TODO: What is criterion? LabelSmoothingCrossEntropy, SoftTargetCrossEntropy differences..
criterion = nn.CrossEntropyLoss()

# TODO: 
#   - set the right number of classes: 4
#   - what is embed_dim?
#   - set drop out rate..
# model_ft = convit_base(pretrained=False, # TODO: How to fine-tune and use a pre-trained?
#                        num_classes=2, 
#                        embed_dim=48,
#                        drop_rate=0.7,
#                        drop_path_rate=0.1,
#                       #  drop_block_rate=None,
#                        depth=12,
#                        local_up_to_layer=12,
#                        locality_strength=1.,
#                        ) # 0.5

model_ft = create_model(
    'convit_base',
    pretrained=False,
    num_classes=2,
    drop_rate=0.7,
    drop_path_rate=0.1,
    drop_block_rate=None,
    local_up_to_layer=12,
    locality_strength=1.,
    embed_dim = 48,
    depth = 12
)


model_ft.to(DEVICE)

# Choose the simple and violent Adam optimizer to reduce the learning rate
optimizer = optim.Adam(model_ft.parameters(), lr=modellr)


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    modellrnew = modellr * (0.1 ** (epoch // 50))
    print("lr:", modellrnew)
    for param_group in optimizer.param_groups:
        param_group['lr'] = modellrnew


In [12]:
# Define training process

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    sum_loss = 0
    total_num = len(train_loader.dataset)
    print(total_num, len(train_loader))
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print_loss = loss.data.item()
        sum_loss += print_loss
        if (batch_idx + 1) % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                       100. * (batch_idx + 1) / len(train_loader), loss.item()))
    ave_loss = sum_loss / len(train_loader)
    print('epoch:{},loss:{}'.format(epoch, ave_loss))


# Verification process
def val(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total_num = len(test_loader.dataset)
    print(total_num, len(test_loader))
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data).to(device), Variable(target).to(device)
            output = model(data)
            loss = criterion(output, target)
            _, pred = torch.max(output.data, 1)
            correct += torch.sum(pred == target)
            print_loss = loss.data.item()
            test_loss += print_loss
        correct = correct.data.item()
        acc = correct / total_num
        avgloss = test_loss / len(test_loader)
        print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            avgloss, correct, len(test_loader.dataset), 100 * acc))
        


# Perform training
for epoch in range(1, EPOCHS + 1):
    adjust_learning_rate(optimizer, epoch)
    train(model_ft, DEVICE, train_loader, optimizer, epoch)
    val(model_ft, DEVICE, test_loader)

# Save the trained model
torch.save(model_ft, 'convit-e25-base-model.pth')

lr: 0.0001
922 29
Train Epoch: 1 [320/922 (34%)]	Loss: 0.642537
Train Epoch: 1 [640/922 (69%)]	Loss: 0.557633
epoch:1,loss:0.6287642439891552
113 4

Val set: Average loss: 2.1112, Accuracy: 85/113 (75%)

lr: 0.0001
922 29
Train Epoch: 2 [320/922 (34%)]	Loss: 0.746065
Train Epoch: 2 [640/922 (69%)]	Loss: 0.405302
epoch:2,loss:0.5179970084593214
113 4

Val set: Average loss: 1.7802, Accuracy: 85/113 (75%)

lr: 0.0001
922 29
Train Epoch: 3 [320/922 (34%)]	Loss: 0.464704
Train Epoch: 3 [640/922 (69%)]	Loss: 0.664858
epoch:3,loss:0.4773795625259136
113 4

Val set: Average loss: 1.5888, Accuracy: 89/113 (79%)

lr: 0.0001
922 29
Train Epoch: 4 [320/922 (34%)]	Loss: 0.332838
Train Epoch: 4 [640/922 (69%)]	Loss: 0.406972
epoch:4,loss:0.43518567444949313
113 4

Val set: Average loss: 1.6998, Accuracy: 87/113 (77%)

lr: 0.0001
922 29
Train Epoch: 5 [320/922 (34%)]	Loss: 0.495749
Train Epoch: 5 [640/922 (69%)]	Loss: 0.415127
epoch:5,loss:0.4338021494191268
113 4

Val set: Average loss: 1.2959, Acc

In [13]:
# Test

# classes = ('Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed',
#            'Common wheat','Fat Hen', 'Loose Silky-bent',
#            'Maize','Scentless Mayweed','Shepherds Purse','Small-flowered Cranesbill','Sugar beet')
# classes = ('Dark', 'Green', 'Light', 'Medium')
classes = (
    'healthy', 
    'leaf_curl'
)

transform_test = transforms.Compose([
  transforms.Resize((224, 224)),
  # transforms.CenterCrop(224),
  transforms.ToTensor(),
  transforms.Normalize([0.4843, 0.5642, 0.3729], [0.1966, 0.1852, 0.1913])  
])
 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# model = torch.load("/content/drive/MyDrive/PROJ518/models/transformer/convit/model.pth")
model = torch.load("/content/drive/MyDrive/PROJ518/models/transformer/convit/convit-e25-base-model.pth")

model.eval()
model.to(DEVICE)
 
# Test 1:
# path = '/content/drive/MyDrive/data/val/Medium/'
# testList = os.listdir(path)
# valid_images = [".jpg",".gif",".png",".tga"]

# for f in testList:
#   print(f)
#   ext = os.path.splitext(f)[1]
#   print(ext)
#   if ext.lower() not in valid_images:
#       continue

#   img = Image.open(os.path.join(path, f))
#   img = transform_test(img)
#   img.unsqueeze_(0)
#   img = Variable(img).to(DEVICE)
#   out = model(img)
  
#   # Predict
#   _, pred = torch.max(out.data, 1)
#   print('Image Name: {0}, predict: {1}'.format(f, classes[pred.data.item()]))


# Test 2:
dataset_test = SeedlingData('/content/drive/MyDrive/PROJ518/Cotton Dataset/data/test', transform_test, test=True)
print(len(dataset_test))

# label of corresponding folder
 
correct = 0
for index in range(len(dataset_test)):
    item = dataset_test[index]
    img, label = item
    img.unsqueeze_(0)
    data = Variable(img).to(DEVICE)
    output = model(data)
    _, pred = torch.max(output.data, 1)
    
    if pred.data.item() == label:
      correct += 1

    print('Image Name: {0} ({1}), predict: {2} -- {3}'.format(dataset_test.imgs[index], label, classes[pred.data.item()], correct))
    index += 1

191
Image Name: /content/drive/MyDrive/PROJ518/Cotton Dataset/data/test/healthy/healthy_128.jpg (0), predict: healthy -- 1
Image Name: /content/drive/MyDrive/PROJ518/Cotton Dataset/data/test/healthy/healthy_213.jpg (0), predict: healthy -- 2
Image Name: /content/drive/MyDrive/PROJ518/Cotton Dataset/data/test/healthy/healthy_140.jpg (0), predict: healthy -- 3
Image Name: /content/drive/MyDrive/PROJ518/Cotton Dataset/data/test/healthy/healthy_37.jpg (0), predict: healthy -- 4
Image Name: /content/drive/MyDrive/PROJ518/Cotton Dataset/data/test/healthy/healthy_43.jpg (0), predict: healthy -- 5
Image Name: /content/drive/MyDrive/PROJ518/Cotton Dataset/data/test/healthy/healthy_67.jpg (0), predict: leaf_curl -- 5
Image Name: /content/drive/MyDrive/PROJ518/Cotton Dataset/data/test/healthy/healthy_139.jpg (0), predict: healthy -- 6
Image Name: /content/drive/MyDrive/PROJ518/Cotton Dataset/data/test/healthy/healthy_30.jpg (0), predict: healthy -- 7
Image Name: /content/drive/MyDrive/PROJ518/Cot